 ### 1. Importing Required Libraries

In [ ]:
import os
import asyncio
from pathlib import Path
from dotenv import load_dotenv
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from semantic_kernel.functions import kernel_function
from typing import Annotated


This section loads key libraries:

- `os`, `asyncio`, `Path`: For file paths, system tasks, and async code.
- `load_dotenv`: Loads environment variables from a `.env` file, helping to keep sensitive credentials (like keys or endpoints) secure and out of your code.
- `DefaultAzureCredential`: Authenticates with Azure services.
- `semantic_kernel`:  Core modules from the Semantic Kernel SDK that allow interaction with AI agents, threads, and kernel plugin functions.
- `Annotated`: Adds descriptions to function parameters.


### 2. Main Function Setup

In [ ]:
async def main():
    os.system('cls' if os.name=='nt' else 'clear')

    # Load the expenses data file
    script_dir = Path(__file__).parent
    file_path = script_dir / 'data.txt'
    with file_path.open('r') as file:
        data = file.read() + "\n"

    # Ask for a prompt
    user_prompt = input(f"Here is the expenses data in your file:\n\n{data}\n\nWhat would you like me to do with the expenses data?\n")
    
    await process_expenses_data(user_prompt, data)

This function:

- Clears the terminal for clean output.
- Loads `data.txt` from the script's folder.
- Shows the file content and prompts the user for a command.
- Calls `process_expenses_data()` to process the prompt and data asynchronously.


### 3. Processing the Expenses Data Using Azure AI Agent

In [ ]:
async def process_expenses_data(prompt, expenses_data):
    # Load environment variables for secure configuration
    load_dotenv()
    ai_agent_settings = AzureAIAgentSettings()

    # Use async context managers for authentication and client creation
    async with (
        DefaultAzureCredential(
            exclude_environment_credential=True,
            exclude_managed_identity_credential=True) as creds,
        AzureAIAgent.create_client(
            credential=creds
        ) as project_client,
    ):
        # Create an AI agent with specific instructions for expense claims
        expenses_agent_def = await project_client.agents.create_agent(
            model= ai_agent_settings.model_deployment_name,
            name="expenses_agent",
            instructions="""You are an AI assistant for expense claim submission.
                            When a user submits expenses data and requests an expense claim, use the plug-in function to send an email to expenses@contoso.com with the subject 'Expense Claim`and a body that contains itemized expenses with a total.
                            Then confirm to the user that you've done so."""
        )

        # Initialize the semantic kernel agent with the EmailPlugin for email simulation
        expenses_agent = AzureAIAgent(
            client=project_client,
            definition=expenses_agent_def,
            plugins=[EmailPlugin()]
        )

        # Create a conversational thread to maintain context for this interaction
        thread: AzureAIAgentThread = AzureAIAgentThread(client=project_client)

        try:
            # Prepare the user prompt and expenses data as messages to send to the agent
            prompt_messages = [f"{prompt}: {expenses_data}"]
            # Await the agent's response asynchronously
            response = await expenses_agent.get_response(thread_id=thread.id, messages=prompt_messages)
            # Print the agent's response to the terminal
            print(f"\n# {response.name}:\n{response}")
        except Exception as e:
            # Catch and print any errors that occur during agent interaction
            print(e)
        finally:
            # Clean up: delete the thread and the agent to free resources
            await thread.delete() if thread else None
            await project_client.agents.delete_agent(expenses_agent.id)


- `load_dotenv()` loads environment variables for credentials and config.  
- `AzureAIAgentSettings()` fetches model deployment info.  
- `DefaultAzureCredential()` provides secure, automatic Azure authentication without hardcoding secrets.  
- `create_client()` initializes the Azure AI Foundry agent client.  
- An agent is created with instructions to submit an expense email using a plugin function.  
- `AzureAIAgentThread()` creates a conversation thread to keep context.  
- The user prompt and data are sent as messages to the agent.  
- The agent responds using the `EmailPlugin`, and the response is printed.  
- Finally, the thread and agent are deleted to clean up resources and prevent leaks.  


### 4. Creating the Email Plugin

In [ ]:
class EmailPlugin:
    # Register this method as a kernel plugin function with a description
    @kernel_function(description="Sends an email.")
    def send_email(self,
                   to: Annotated[str, "Who to send the email to"],
                   subject: Annotated[str, "The subject of the email."],
                   body: Annotated[str, "The text body of the email."]):
        # Simulate sending email by printing the details to the console
        print("\nTo:", to)
        print("Subject:", subject)
        print(body, "\n")


Plugin: 
A plugin is a modular piece of code that adds specific functionality to the agent, which can be called during its execution.

Decorator (@kernel_function): 
A decorator is a Python feature that modifies or enhances a function’s behavior; here it registers `send_email` as a callable plugin for the Semantic Kernel agent.

Annotated:
The `Annotated` type adds descriptions to function parameters, helping the AI understand what each argument represents.

`send_email` method:
This method simulates email sending by printing the email content.

### 5. Running the Script

In [ ]:
if __name__ == "__main__":
    asyncio.run(main())